In [1]:
import os
import numpy as np
from pandas import read_csv
from tqdm import tqdm

## Section 1: Task performance metrics

### 1.1 Compute permutation statistics

In [2]:
from scipy.stats import ttest_ind
np.random.seed(47404)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define variables of interest.
cols = ['accuracy','points','ws','ls','perseveration','beta','eta_p','eta_n','kappa']

## Define number of bootstraps.
n_iter = 5000

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Permutation statistics.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load and prepare data.
metrics = read_csv(os.path.join('data', 'metrics.csv'))
correlates = read_csv(os.path.join('data', 'correlates.csv'))

## Merge DataFrames.
data = correlates.merge(metrics[['subject','platform','infreq']])
data = data.set_index('infreq')

## Binarize infrequency failures.
data.index = np.where(data.index > 0, 1, 0)

## Restrict to relevant columns.
data = data[cols]

## Compute means by group.
gb = data.groupby(level=0).mean().T

## Compute observed statistics.
t_obs = np.zeros(len(data.columns))
for i, col in enumerate(data.columns):
    t_obs[i], _ = ttest_ind(data.loc[0, col], data.loc[1, col])
    
## Compute null statistics.
t_null = np.zeros((n_iter, t_obs.size))
for n in tqdm(range(n_iter)):
    
    ## Permute labels.
    data.index = np.random.permutation(data.index)
    
    ## Compute null statistics.
    for i, col in enumerate(data.columns):
        t_null[n,i], _ = ttest_ind(data.loc[0, col], data.loc[1, col])
        
## Compute p-values.
p_val = np.mean(np.abs(t_obs) < np.abs(t_null), axis=0)

## Add to table.
gb['tval'] = t_obs
gb['pval'] = p_val

100%|██████████| 5000/5000 [00:56<00:00, 88.64it/s] 


### 1.2 Supplementary tables

In [3]:
## Update columns.
gb = gb.rename(columns={0: 'Attentive', 1:'C/IE'})

## Update rows.
gb = gb.rename(index={'accuracy': 'Acc', 'points': 'Pts', 'ws': 'WS', 'ls': 'LS',
                      'perseveration': 'Pers', 'beta': r'$\beta$', 'eta_p': r'$\eta_p$', 
                      'eta_n': r'$\eta_n$', 'kappa': r'$\kappa$'})

## Convert to LaTeX.
print(gb.round(3).to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &  Attentive &    C/IE &   tval &   pval \\
\midrule
Acc      &      0.587 &   0.532 &  4.008 &  0.000 \\
Pts      &     50.163 &  47.729 &  2.376 &  0.019 \\
WS       &      0.898 &   0.776 &  5.387 &  0.000 \\
LS       &      0.609 &   0.751 & -5.335 &  0.000 \\
Pers     &      0.245 &   0.259 & -1.505 &  0.139 \\
\$\textbackslash beta\$  &      6.754 &   4.082 &  5.404 &  0.000 \\
\$\textbackslash eta\_p\$ &      0.643 &   0.551 &  2.846 &  0.007 \\
\$\textbackslash eta\_n\$ &      0.738 &   0.784 & -1.516 &  0.120 \\
\$\textbackslash kappa\$ &     -0.069 &  -0.218 &  3.729 &  0.000 \\
\bottomrule
\end{tabular}



<ipython-input-3-401e5fbcf299>:10: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(gb.round(3).to_latex())
